In [3]:
import pandas as pd
import numpy as np

In [4]:
impressions_df_01 = pd.read_csv('/home/sk2159/Downloads/c48cb4cb-46c9-48ae-b2fd-df2ff244fbc8.csv')
impressions_df_02 = pd.read_csv('/home/sk2159/Downloads/6981025e-b785-43a4-8856-12fe16495a4e.csv')
impressions_df_03 = pd.read_csv('/home/sk2159/Downloads/fbdbbe06-bca3-40b9-a85b-0aa494544754.csv')

impressions = [impressions_df_01, impressions_df_02, impressions_df_03]

# impressions_df = pd.concat([impressions_df_01, impressions_df_02, impressions_df_03])

In [5]:
for impressions_df in impressions
    impressions_df["true_is_iab_inview"] = impressions_df.apply(
        lambda x: 
        True if (
            (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
        ) | (x["is_iab_inview"] == True) 
        else False, 
        axis=1
    )

In [6]:
# run through predictions for 2022
import os
import json

def _get_pred_map():
    pred_map = dict()
    
    #for i in range(1, 7):
    #for i in range(7, 13):
    for i in range(1, 4):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../predictions/2023/{i}'):
            for h in os.listdir(f'../predictions/2023/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../predictions/2023/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [7]:
def _get_final_fixation(is_iab_inview, gaze_valid, is_fixated, id_, pred_map):
    """
    Do we need to remove 
        gaze_valid = True (eyetracker configured/enabled)
        is_fixated = True (gaze cordinates and ad overlap)
        is_iab_inview = False (example, only 40% of ad is visable)
    """
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:# and is_iab_inview:
        return pred_map[id_]
    return False


impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['true_is_iab_inview'], row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)

In [12]:
final_df = impressions_df.groupby("part_month").apply(
    lambda x: pd.Series(
        {
            "impressions": x["id"].count(),""
            "fixations (no predictions)": x[x["is_fixated"] == True]["id"].count(),
            "fixations (predictions)": x[x["final_fixation"] == True]["id"].count()
        }
    )
)
final_df = final_df.reset_index()
final_df

,part_month,impressions,fixations (no predictions),fixations (predictions)
0,1,390444,26973,41418
1,2,365605,24184,36455
2,3,406535,30566,38204


In [19]:
for index, series in final_df.iterrows():
    print(f"month: {series['part_month']}")
    print(f"impressions: {series['impressions']}")
    print(f"fixations (no predictions): {series['fixations (no predictions)']}")
    print(f"final:fixations (predictions): {series['fixations (predictions)']}")
    print("\n")

month: 1
impressions: 390444
fixations (no predictions): 26973
final:fixations (predictions): 41418


month: 2
impressions: 365605
fixations (no predictions): 24184
final:fixations (predictions): 36455


month: 3
impressions: 406535
fixations (no predictions): 30566
final:fixations (predictions): 38204




In [9]:
print(f"impressions: {impressions_df['id'].count()}")
print(f"fixations (no predictions): {impressions_df[impressions_df['is_fixated'] == True]['id'].count()}")
print(f"final:fixations (predictions): {impressions_df[impressions_df['final_fixation'] == True]['id'].count()}")

impressions: 1162584
fixations (no predictions): 81723
final:fixations (predictions): 116077
